# Assignment 4: Data Collection

This work aims to automate the collection of real estate data from CoinAfrique using Selenium and BeautifulSoup.

In [1]:
# import packages
import pandas as pd
from selenium import webdriver 
from selenium.webdriver.common.by import By 
from bs4 import BeautifulSoup as bs


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/usr/lib/python3/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/lib/python3/dist-packages/traitlets/config/application.py", line 846, in launch_instance
    app.start()
  File "/usr/lib/python3/dist-packages/ipykernel/kernelapp.py", line 677, in start
    s

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/usr/lib/python3/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/lib/python3/dist-packages/traitlets/config/application.py", line 846, in launch_instance
    app.start()
  File "/usr/lib/python3/dist-packages/ipykernel/kernelapp.py", line 677, in start
    s

AttributeError: _ARRAY_API not found

In [2]:
# instantiate a Chrome options object
options = webdriver.ChromeOptions() 
# set the options to use Chrome in headless mode (used for running the script in the background)
options.add_argument("--headless=new") 
# initialize an instance of the Chrome driver (browser) in headless mode
driver = webdriver.Chrome(options=options)

## scrape the following informations

In [3]:
df = pd.DataFrame()
data = []
for index in range(1, 51):
    url = f'https://ci.coinafrique.com/categorie/appartements?page={index}'
    # get the url 
    driver.get(url)
    # get the content 
    soup = bs(driver.page_source, 'html.parser')
    # find containers 
    containers = soup.find_all('div', 'col s6 m4 l3')
    for container in containers:
        
        # Retrieve the URL of the subpage: search for the link in the container,
        # extract its attribute, and thus access the detailed page.
        url_sous_page = 'https://ci.coinafrique.com' + container.find('a')['href']
        driver.get(url_sous_page)
        res_sous_page = driver.page_source
        soup_page = bs(res_sous_page, 'html.parser')
        
        #Extract a specific table from the sous-page.
        infos = soup_page.find('div', 'details-characteristics')
        
        try:
            
            info = infos.find_all('li')
            # scrape the price, price, adress  
            details = container.find('p', class_ = 'ad__card-description').text 
            price = container.find('p', class_ ='ad__card-price').text.replace("CFA", "")  
            adress = container.find('p', class_ = 'ad__card-location').span.text

            # scraper the numbers of bathrooms, rooms and area
            num_of_bathrooms = info[0].find_all('span')[1].text
            num_of_rooms = info[1].find_all('span')[1].text
            area = "".join(info[2].find_all('span')[1].text.replace('m2', ''))

            #image link
            image_links = 'https://ci.coinafrique.com' + container.find('a')['href']
            
            # 
            dic = {
                'details':details,
                'price':price,
                'adress':adress,
                'num_of_bathrooms':num_of_bathrooms,
                'num_of_rooms':num_of_rooms,
                'area':area,
                'image_links':image_links
            }
            
            data.append(dic)

        except:
            pass
# All extracted data
data_scrape = pd.DataFrame(data)
df = pd.concat([df, data_scrape], axis = 0).reset_index(drop = True)

In [8]:
df.head()

,details,price,adress,num_of_bathrooms,num_of_rooms,area,image_links
0,Location appartements 4pièces - Cocody Vallon,1 800 000,"Cocody, Abidjan, Côte d'Ivoire",4,3,210,https://ci.coinafrique.com/annonce/appartement...
1,Location appartements 4pièces -Cocody Riviera ...,650 000,"Cocody, Abidjan, Côte d'Ivoire",4,4,400,https://ci.coinafrique.com/annonce/appartement...
2,Vente appartement 3pièces -Cocody,80 000 000,"Cocody, Abidjan, Côte d'Ivoire",3,3,100,https://ci.coinafrique.com/annonce/appartement...
3,Location appartement 4 pièces à Marcory-champroux,130 000 000,"Abidjan, Côte d'Ivoire",4,2,115,https://ci.coinafrique.com/annonce/appartement...
4,Location appartement standing 2 pièces - Cocod...,300 000,"Cocody, Abidjan, Côte d'Ivoire",2,1,60,https://ci.coinafrique.com/annonce/appartement...


In [9]:
df.shape

(2949, 7)

## Data Cleanning

In [15]:
# Checking columns
df.columns

Index(['details', 'price', 'adress', 'num_of_bathrooms', 'num_of_rooms',
       'area', 'image_links'],
      dtype='object')

In [10]:
# Checking the missing values
df.isnull().sum()

details             0
price               0
adress              0
num_of_bathrooms    0
num_of_rooms        0
area                0
image_links         0
dtype: int64

In [11]:
# Checking the duplicates
df.duplicated().sum()

np.int64(8)

In [13]:
# Drop the duplicates
df.drop_duplicates(inplace = True)

In [14]:
#checking the types
df.dtypes

details             object
price               object
adress              object
num_of_bathrooms    object
num_of_rooms        object
area                object
image_links         object
dtype: object

In [28]:
# Change the types
df['num_of_bathrooms'] = df['num_of_bathrooms'].astype(int)
df['num_of_rooms'] = df['num_of_rooms'].astype(int)
df['area'] = df['area'].astype(float)

/tmp/ipykernel_265129/2067144214.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['num_of_bathrooms'] = df['num_of_bathrooms'].astype(int)
/tmp/ipykernel_265129/2067144214.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['num_of_rooms'] = df['num_of_rooms'].astype(int)
/tmp/ipykernel_265129/2067144214.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

In [18]:
df['price'] = df['price'].str.strip()

In [19]:
df['price'].unique()

array(['1 800 000', '650 000', '80 000 000', '130 000 000', '300 000',
       '500 000', '700 000', '45 000 000', '550 000', '800 000',
       '1 600 000', '1 300 000', '250 000 000', '400 000', '600 000',
       '1 500 000', '65 000 000', '145 000 000', '350 000', '140 000',
       '125 000 000', '85 000 000', '200 000', '119 000 000',
       '95 000 000', '200 000 000', '420 000', '1 000 000', '1 200 000',
       '750 000', '320 000', '140 000 000', '850 000', '1 100 000',
       '50 000 000', '2 000 000', '1 750 000', '1 650 000', '60 000 000',
       '1 550 000', '150 000 000', '950 000', '900 000', '250 000',
       '35 000 000', '210 000', '70 000 000', '170 000', '160 000 000',
       '180 000 000', '445 000', '38 000 000', '230 000',
       'Prix sur demande', '190 000 000', '230 000 000', '75 000 000',
       '120 000 000', '110 000', '55 000 000', '2 500 000', '450 000',
       '47 000 000', '40 000 000', '380 000', '90 000 000', '475 000',
       '430 000', '37 000 000', '32

In [20]:
df = df[df['price'] != 'Prix sur demande']

In [25]:
df['price'] = df['price'].str.replace(' ', '')

/tmp/ipykernel_265129/1145929816.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price'] = df['price'].str.replace(' ', '')


In [26]:
df['price'] = df['price'].astype(int)

/tmp/ipykernel_265129/2468783641.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price'] = df['price'].astype(int)


In [29]:
# Checking the types
df.dtypes

details              object
price                 int64
adress               object
num_of_bathrooms      int64
num_of_rooms          int64
area                float64
image_links          object
dtype: object

In [30]:
df.head()

,details,price,adress,num_of_bathrooms,num_of_rooms,area,image_links
0,Location appartements 4pièces - Cocody Vallon,1800000,"Cocody, Abidjan, Côte d'Ivoire",4,3,210.0,https://ci.coinafrique.com/annonce/appartement...
1,Location appartements 4pièces -Cocody Riviera ...,650000,"Cocody, Abidjan, Côte d'Ivoire",4,4,400.0,https://ci.coinafrique.com/annonce/appartement...
2,Vente appartement 3pièces -Cocody,80000000,"Cocody, Abidjan, Côte d'Ivoire",3,3,100.0,https://ci.coinafrique.com/annonce/appartement...
3,Location appartement 4 pièces à Marcory-champroux,130000000,"Abidjan, Côte d'Ivoire",4,2,115.0,https://ci.coinafrique.com/annonce/appartement...
4,Location appartement standing 2 pièces - Cocod...,300000,"Cocody, Abidjan, Côte d'Ivoire",2,1,60.0,https://ci.coinafrique.com/annonce/appartement...


## Data recording

In [31]:
df.to_csv('data_clean.csv', index = False)